In [5]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind

# Load your data
df = pd.read_csv("C:/Users/Student/Documents/kaim documents/week 3/MachineLearningRating_v3/MachineLearningRating_v3.txt", delimiter='|')

# Clean column names
df.columns = df.columns.str.strip().str.lower()
print(df.columns)

# Create binary flag for claim (Claim Frequency)
df['had_claim'] = df['totalclaims'] > 0

# Claim Severity (only for those with claims)
df_claims_only = df[df['had_claim'] == True]

# Margin
df['margin'] = df['totalpremium'] - df['totalclaims']


C:\Users\Student\AppData\Local\Temp\ipykernel_34068\3713882454.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Student/Documents/kaim documents/week 3/MachineLearningRating_v3/MachineLearningRating_v3.txt", delimiter='|')


Index(['underwrittencoverid', 'policyid', 'transactionmonth',
       'isvatregistered', 'citizenship', 'legaltype', 'title', 'language',
       'bank', 'accounttype', 'maritalstatus', 'gender', 'country', 'province',
       'postalcode', 'maincrestazone', 'subcrestazone', 'itemtype', 'mmcode',
       'vehicletype', 'registrationyear', 'make', 'model', 'cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'numberofdoors',
       'vehicleintrodate', 'customvalueestimate', 'alarmimmobiliser',
       'trackingdevice', 'capitaloutstanding', 'newvehicle', 'writtenoff',
       'rebuilt', 'converted', 'crossborder', 'numberofvehiclesinfleet',
       'suminsured', 'termfrequency', 'calculatedpremiumperterm',
       'excessselected', 'covercategory', 'covertype', 'covergroup', 'section',
       'product', 'statutoryclass', 'statutoryrisktype', 'totalpremium',
       'totalclaims'],
      dtype='object')


In [6]:
contingency = pd.crosstab(df['province'], df['had_claim'])
chi2, p, _, _ = chi2_contingency(contingency)
print(f"Province vs Claim Frequency p-value: {p:.4f}")


Province vs Claim Frequency p-value: 0.0000


In [12]:
province_A = df_claims_only[df_claims_only['province'] == 'Western Cape']['totalclaims']
province_B = df_claims_only[df_claims_only['province'] == 'Gauteng']['totalclaims']

t_stat, p = ttest_ind(province_A, province_B, equal_var=False)
print(f"Province vs Claim Severity p-value: {p:.4f}")


Province vs Claim Severity p-value: 0.0306


In [15]:
# Filter claim severity for two postal codes
zip_A = df_claims_only[df_claims_only['postalcode'] == 2000]['totalclaims']
zip_B = df_claims_only[df_claims_only['postalcode'] == 122]['totalclaims']

# Check sizes
print("Group A (Postal Code 2000) count:", len(zip_A))
print("Group B (Postal Code 122) count:", len(zip_B))

# Run t-test only if both groups have enough data
if len(zip_A) >= 2 and len(zip_B) >= 2:
    from scipy.stats import ttest_ind
    t_stat, p = ttest_ind(zip_A, zip_B, equal_var=False)
    print(f"Postal Code vs Claim Severity p-value: {p:.4f}")
    
    # Optional: interpret the result
    if p < 0.05:
        print("✅ Reject the null hypothesis — significant difference in claim severity between postal codes.")
    else:
        print("❌ Fail to reject the null — no significant difference.")
else:
    print("⚠️ Not enough data in one of the groups for valid t-test.")




Group A (Postal Code 2000) count: 486
Group B (Postal Code 122) count: 210
Postal Code vs Claim Severity p-value: 0.7002
❌ Fail to reject the null — no significant difference.


In [10]:
print(zip_A.shape, zip_B.shape)

(0,) (0,)


In [13]:
# Show top zip codes where there are claims
print(df_claims_only['postalcode'].value_counts().head(10))


postalcode
2000    486
122     210
299      67
8000     51
7784     50
1863     44
470      44
400      34
458      32
2196     32
Name: count, dtype: int64


In [16]:
contingency_gender = pd.crosstab(df['gender'], df['had_claim'])
chi2, p, _, _ = chi2_contingency(contingency_gender)
print(f"Gender vs Claim Frequency p-value: {p:.4f}")


Gender vs Claim Frequency p-value: 0.0266


In [17]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind

# Load your data


# Create derived columns
df['Had_Claim'] = df['totalclaims'] > 0
df['margin'] = df['totalpremium'] - df['totalclaims']

# === Gender-based Analysis ===
group_male = df[df['gender'] == 'Male']
group_female = df[df['gender'] == 'Female']

# Claim Frequency (Chi-squared)
claim_freq_table = pd.crosstab(df['gender'], df['Had_Claim'])
chi2, p_freq_gender, _, _ = chi2_contingency(claim_freq_table)

# Claim Severity (T-test)
severity_m = group_male[group_male['totalclaims'] > 0]['totalclaims']
severity_f = group_female[group_female['totalclaims'] > 0]['totalclaims']
_, p_sev_gender = ttest_ind(severity_m, severity_f, equal_var=False)

# Margin (T-test)
_, p_margin_gender = ttest_ind(group_male['margin'], group_female['margin'], equal_var=False)

# === Postal Code-based Analysis ===
df_claims_only = df[df['totalclaims'] > 0]
top_postals = df_claims_only['postalcode'].value_counts().head(10).index.tolist()
zip_A = df_claims_only[df_claims_only['postalcode'] == top_postals[0]]['totalclaims']
zip_B = df_claims_only[df_claims_only['postalcode'] == top_postals[1]]['totalclaims']

if len(zip_A) >= 2 and len(zip_B) >= 2:
    _, p_postal = ttest_ind(zip_A, zip_B, equal_var=False)
else:
    p_postal = np.nan

# === Province-based Analysis ===
claim_freq_province = pd.crosstab(df['province'], df['Had_Claim'])
_, p_freq_province, _, _ = chi2_contingency(claim_freq_province)

# === Summary Output ===
results = {
    "Claim Frequency by Gender (p)": p_freq_gender,
    "Claim Severity by Gender (p)": p_sev_gender,
    "Margin by Gender (p)": p_margin_gender,
    "Claim Severity by Postal Code (p)": p_postal,
    "Claim Frequency by Province (p)": p_freq_province
}

for key, value in results.items():
    print(f"{key}: {value:.4f}" if pd.notna(value) else f"{key}: Not enough data")


Claim Frequency by Gender (p): 0.0266
Claim Severity by Gender (p): 0.5680
Margin by Gender (p): 0.8015
Claim Severity by Postal Code (p): 0.7002
Claim Frequency by Province (p): 0.0000
